In [1]:
# https://github.com/patrickloeber/pytorchTutorial/blob/master/09_dataloader.py

import math
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

In [2]:
# graident computation etc. not efficient for whole dataset
# -> divide dataset into small batches

"""
# training loop
for epoch in range(num_epochs):
    # loop over all batches
    for i in range(total_batches):
        batch_x, batch_y = ...
"""

'\n# training loop\nfor epoch in range(num_epochs):\n    # loop over all batches\n    for i in range(total_batches):\n        batch_x, batch_y = ...\n'

In [3]:
# epoch = one forward and backward pass of ALL training samples
# batch_size = number of training samples used in one forward/backward pass
# number of iterations = number of passes, each pass (forward + backward) using [batch_size] number of shapes
# e.g. 100 samples, batch_size=20 -> 100/20 = 5 iterations for 1 epoch

# --> DataLoader can do the batch computation for us

# Implement a custom Dataset:
# inherit Dataset
# implement __init__, __getitem__, and __len__

class WineDataset(Dataset):
    def __init__(self):
        # initialize data, download, etc.
        # read with numpy or pandas
        # skiprows=1 means skip the first row, which is the header
        xy = np.loadtxt("ds/wine.csv", delimiter=",", dtype=np.float32, skiprows=1)
        # the 1st dimension is the # of samples
        self.n_samples = xy.shape[0]
        
        # split data
        # here the first column is the class label, the rest are the features
        # get all samples (:), get everything but the first column (1:) 
        self.x_data = torch.from_numpy(xy[:, 1:]) # size [n_samples, n_features]
        # get all samples (:), and only the very first column ([0])
        self.y_data = torch.from_numpy(xy[:, [0]]) # size [n_samples, 1]
    
    # support indexing such that dataset[i] can be used to get i-th sample
    # tl;dr returns a tuple
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
# create dataset
dataset = WineDataset()

In [4]:
# get first sample and unpack
first_data = dataset[0]
features, labels = first_data # i guess this assignment operation is automatic?
print(features, labels)

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [5]:
# load whole dataset with DataLoader
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses
# !!! IF YOU GET AN ERROR DURING LOADING, SET num_workers TO 0 !!!
train_loader = DataLoader(dataset=dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=0) # 2

In [6]:
# convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = next(dataiter)
features, labels = data
print(features, labels)

tensor([[1.3030e+01, 9.0000e-01, 1.7100e+00, 1.6000e+01, 8.6000e+01, 1.9500e+00,
         2.0300e+00, 2.4000e-01, 1.4600e+00, 4.6000e+00, 1.1900e+00, 2.4800e+00,
         3.9200e+02],
        [1.3750e+01, 1.7300e+00, 2.4100e+00, 1.6000e+01, 8.9000e+01, 2.6000e+00,
         2.7600e+00, 2.9000e-01, 1.8100e+00, 5.6000e+00, 1.1500e+00, 2.9000e+00,
         1.3200e+03],
        [1.3770e+01, 1.9000e+00, 2.6800e+00, 1.7100e+01, 1.1500e+02, 3.0000e+00,
         2.7900e+00, 3.9000e-01, 1.6800e+00, 6.3000e+00, 1.1300e+00, 2.9300e+00,
         1.3750e+03],
        [1.3900e+01, 1.6800e+00, 2.1200e+00, 1.6000e+01, 1.0100e+02, 3.1000e+00,
         3.3900e+00, 2.1000e-01, 2.1400e+00, 6.1000e+00, 9.1000e-01, 3.3300e+00,
         9.8500e+02]]) tensor([[2.],
        [1.],
        [1.],
        [1.]])


In [7]:
# dummy training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples / 4)
print(total_samples, n_iterations)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # here: 178 samples, batch_size = 4, n_iters = 178 / 4 = 44.5 -> 45 iterations
        # run your training process
        if (i + 1) % 5 == 0:
            print(f"Epoch: {epoch + 1}/{num_epochs}, Step {i + 1}/{n_iterations}| inputs {inputs.shape} | labels {labels.shape}")
            
# some famous datasets are available in torchvision.datasets
# e.g. MNIST, Fashion-MNIST, CIFAR10, COCO

train_dataset = torchvision.datasets.MNIST(root="./data",
                                           train=True,
                                           transform=torchvision.transforms.ToTensor(),
                                           download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=3, shuffle=True)

178 45
Epoch: 1/2, Step 5/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 10/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 15/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 20/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 25/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 30/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 35/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 40/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 1/2, Step 45/45| inputs torch.Size([2, 13]) | labels torch.Size([2, 1])
Epoch: 2/2, Step 5/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 2/2, Step 10/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 2/2, Step 15/45| inputs torch.Size([4, 13]) | labels torch.Size([4, 1])
Epoch: 2/2, Step 20/45| inputs torch.Size([4, 1

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%
15.9%

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%
100.0%

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [8]:
# look at one random sample
dataiter = iter(train_loader)
data = next(dataiter)
inputs, targets = data
print(inputs.shape, targets.shape)

torch.Size([3, 1, 28, 28]) torch.Size([3])
